In [ ]:
import numpy as np
import pylab
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import MyMultiSimTools as mst



In [ ]:
NuisCov=np.loadtxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Matrices/NuisanceCovariance.txt")

In [ ]:
Modes=4

In [ ]:
Amp1sig=np.loadtxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/Widths/AmpWidths.txt")
Phs1sig=np.loadtxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Constraints/Widths/PhsWidths.txt")

AmpWidths=Amp1sig[0:Modes+1,3]-Amp1sig[0:Modes+1,1]
PhsWidths=Phs1sig[0:Modes,3]-Phs1sig[0:Modes,1]

AmpCentral=Amp1sig[0:Modes+1,1]
PhsCentral=Amp1sig[0:Modes,1]

amp_modes=range(0,Modes+1)
phs_modes=range(1,Modes+1)

In [ ]:
bestfit= mst.GetIcemodel( central, 
                                    amp_modes_to_shift  = amp_modes, 
                                    amp_shifts          = AmpCentral,
                                    phs_modes_to_shift  = phs_modes,
                                    phs_shifts          = PhsCentral)

mode0pert= mst.GetIcemodel( central, 
                                    amp_modes_to_shift  = [0], 
                                    amp_shifts          = [AmpWidths[0]])

In [ ]:
icepath = "./Dima_Icemodel.dat"
central = mst.LoadIcedata(icepath)
depth=1948-central[0]
outdir = "./models/"

ModelsToRun=5000
AllModels=[]
for i in range(0,ModelsToRun):
    Realization=np.random.multivariate_normal(np.zeros(9),NuisCov)
    amp_shifts=Realization[0:5]*AmpWidths
    phs_shifts=Realization[5:9]*PhsWidths
    
    pert_amp = mst.GetIcemodel( central, 
                                    amp_modes_to_shift  = amp_modes, 
                                    amp_shifts          = amp_shifts,
                                    phs_modes_to_shift  = phs_modes,
                                    phs_shifts          = phs_shifts)
    AllModels.append(pert_amp[1])
    if (i%100==0):
        print i


AllModes=np.array(AllModels)

In [ ]:
freq68=[]
freq95=[]
freq99=[]
AbsFracShift=[]
TopLine=1000
pylab.figure(figsize=(6,3.5),dpi=300)
for i in range(0,TopLine):
    AbsFracShift.append(np.abs(AllModels[i]/central[1]-1))
    
for fs in np.transpose(AbsFracShift):
    freq68.append(sorted(fs)[int(len(fs)*0.68)])
    freq95.append(sorted(fs)[int(len(fs)*0.95)])
    freq99.append(sorted(fs)[int(len(fs)*0.99)])
SDs=[]
for i in np.transpose(AllModes):
    SDs.append(np.std(i))
    

for i in range(0, TopLine):
    pylab.plot(depth,AbsFracShift[i],alpha=0.02,color='blue',linewidth=1)
pylab.plot(depth,freq68,'-',color='red',linewidth=3,label='68 %')
pylab.plot(depth,freq95,'--',color='red',linewidth=3,label='95 %')
pylab.plot(depth,freq99,':',color='red',linewidth=3,label='99 %')

    #pylab.errorbar(central[0],Means/central[1], yerr=SDs/central[1],fmt='o')
pylab.ylim(0,0.5)
pylab.xlim(-700,700)
pylab.ylim()
pylab.ylabel("Absolute fractional shift")
pylab.xlabel("Depth / m"),
pylab.title("Including mode correlations")
pylab.legend(loc='upper center',ncol=3,title='Envelopes')
pylab.tight_layout()
pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Envelope/AbsFracShift.png",dpi=300,bbox_inches='tight')

In [ ]:
pylab.figure(figsize=(6,3.5),dpi=300)



pylab.plot(depth, freq68,color='DarkRed',alpha=1,label=r'MultiSim, modes 0-4, Freq 68%',linewidth=2)
pylab.plot(depth, SDs/central[1],'--',color='DarkRed',alpha=1,label=r'MultiSim, modes 0-4, SD $1\sigma$',linewidth=2)

#pylab.plot(depth ,bestfit[1]/central[1]-1.,color='DarkRed',label=r'MultiSim 4-mode best fit')


pylab.plot(depth,1.-mode0pert[1]/central[1],color='DarkBlue',alpha=1.0,label='Multisim, mode 0 only',linewidth=2)
pylab.plot(depth,np.ones_like(bestfit[1])*0.1,color='dimgrey',alpha=1.0,label='SpiceMie Ice Ellipse',linewidth=2)


#pylab.ylim(0,0.25)
pylab.xlim(-700,700)
pylab.ylim(0,0.2)
pylab.legend(loc='upper center',ncol=1,title='Uncertainty Comparison')
pylab.ylabel("Absolute Fracitonal Shift")
pylab.xlabel("Depth / m")
pylab.tight_layout()
pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Envelope/ModelComparison.png",dpi=300,bbox_inches='tight')

In [ ]:
np.diag(np.diag(NuisCov))

In [ ]:
icepath = "./Dima_Icemodel.dat"
central = mst.LoadIcedata(icepath)
depth=1948-central[0]
outdir = "./models/"

ModelsToRun=5000
AllModels=[]
DiagCov=np.diag(np.diag(NuisCov))
for i in range(0,ModelsToRun):
    Realization=np.random.multivariate_normal(np.zeros(9),DiagCov)
    amp_shifts=Realization[0:5]*AmpWidths
    phs_shifts=Realization[5:9]*PhsWidths
    
    pert_amp = mst.GetIcemodel( central, 
                                    amp_modes_to_shift  = amp_modes, 
                                    amp_shifts          = amp_shifts,
                                    phs_modes_to_shift  = phs_modes,
                                    phs_shifts          = phs_shifts)
    AllModels.append(pert_amp[1])
    if (i%100==0):
        print i


AllModes=np.array(AllModels)

In [ ]:
freq68=[]
freq95=[]
freq99=[]
AbsFracShift=[]
TopLine=1000
pylab.figure(figsize=(6,3.5),dpi=300)
for i in range(0,TopLine):
    AbsFracShift.append(np.abs(AllModels[i]/central[1]-1))
    
for fs in np.transpose(AbsFracShift):
    freq68.append(sorted(fs)[int(len(fs)*0.68)])
    freq95.append(sorted(fs)[int(len(fs)*0.95)])
    freq99.append(sorted(fs)[int(len(fs)*0.99)])
SDs=[]
for i in np.transpose(AllModes):
    SDs.append(np.std(i))
    

for i in range(0, TopLine):
    pylab.plot(central[0],AbsFracShift[i],alpha=0.02,color='blue',linewidth=1)
pylab.plot(central[0],freq68,'-',color='red',linewidth=3,label='68 %')
pylab.plot(central[0],freq95,'--',color='red',linewidth=3,label='95 %')
pylab.plot(central[0],freq99,':',color='red',linewidth=3,label='99 %')

    #pylab.errorbar(central[0],Means/central[1], yerr=SDs/central[1],fmt='o')
pylab.ylim(0,0.5)
pylab.xlim(1200,2700)
pylab.ylim()
pylab.ylabel("Absolute fractional shift")
pylab.xlabel("Depth / m"),
pylab.legend(loc='upper center',ncol=3,title='Envelopes')
pylab.tight_layout()
pylab.title("Neglecting mode correlations")
pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Envelope/AbsFracShiftNoCorr.png",dpi=300,bbox_inches='tight')